In [166]:
import numpy as np
import pandas as pd

(sun0, mon0, tue0, wed0, thu0, fri0, sat0, sun1, mon1, tue1, wed1, thu1, fri1, sat1) = range(14)
labels = ['RG', 'LA', 'LS', 'CD', 'CN', 'LH', 'LN', 'MB']
dlabels = ['sun0', 'mon0', 'tue0', 'wed0', 'thu0', 'fri0', 'sat0', 
           'sun1', 'mon1', 'tue1', 'wed1', 'thu1', 'fri1', 'sat1']
days = dict(zip(range(14), dlabels))
row_names = dict(zip(range(8), labels))
row_names[8] = "Total"

def print_by_lv_cat(df):
    if df.loc['RG'].sum() > 0: print(f"RG: {df.loc['RG'].sum()}") 
    if df.loc['LA'].sum() > 0: print(f"LA: {df.loc['LA'].sum()}")
    if df.loc['LS'].sum() > 0: print(f"LS: {df.loc['LS'].sum()}")
    if df.loc['CD'].sum() > 0: print(f"CD: {df.loc['CD'].sum()}")
    if df.loc['CN'].sum() > 0: print(f"CN: {df.loc['CN'].sum()}")
    if df.loc['LH'].sum() > 0: print(f"LH: {df.loc['LH'].sum()}")
    if df.loc['LN'].sum() > 0: print(f"LN: {df.loc['LN'].sum()}")
    if df.loc['MB'].sum() > 0: print(f"MB: {df.loc['MB'].sum()}")
    print("")

def sum_by_leave_category(df):
    rgs = df.loc['RG'].sum()
    las = df.loc['LA'].sum()
    lss = df.loc['LS'].sum()
    cds = df.loc['CD'].sum()
    cns = df.loc['CN'].sum()
    lhs = df.loc['LH'].sum()
    lns = df.loc['LN'].sum()
    mbs = df.loc['MB'].sum()
    sums = pd.Series([rgs, las, lss, cds, cns, lhs, lns, mbs], index=labels)
    return sums

def sum_by_days(df):
    day_sums = pd.Series([df['sun0'].sum(), df['mon0'].sum(), df['tue0'].sum(), df['wed0'].sum(), df['thu0'].sum(), df['fri0'].sum(), df['sat0'].sum(),
                          df['sun1'].sum(), df['mon1'].sum(), df['tue1'].sum(), df['wed1'].sum(), df['thu1'].sum(), df['fri1'].sum(), df['sat1'].sum()],
                        index=dlabels)
    return day_sums

def rg(day, val):
    df.loc['RG', days[day]] = val
    
def la(day, val):
    df.loc['LA', days[day]] = val
    
def ls(day, val):
    df.loc['LS', days[day]] = val
    
def cd(day, val):
    df.loc['CD', days[day]] = val
    
def cn(day, val):
    df.loc['CN', days[day]] = val
    
def lh(day, val):
    df.loc['LH', days[day]] = val
    
def ln(day, val):
    df.loc['LN', days[day]] = val
    
def mb(day, val):
    df.loc['MB', days[day]] = val    
        
def print_leave(leave, msg=''):    
    print("{0:20s}  {1:6}    {2:6}    {3:6}".format(msg, leave[0], leave[1], leave[2]))
    
def print_leave_summary(leave):
    total_leave = leave[0] + leave[2]
    al_in_days = total_leave // 8
    sl_in_days = leave[1] // 8
    al_remainder = total_leave % 8
    sl_remainder = leave[1] % 8
    print("                       {0:2} d {1} hrs     {2:2} d {3} hrs".
          format(al_in_days, al_remainder, sl_in_days, sl_remainder))
    
def print_usable_leave(leave):
    free_to_use = (leave[0] - 240) + leave[2]
    usable_days = free_to_use // 8
    usable_hrs = free_to_use % 8
    print(f'Usable leave: {usable_days:2} d {usable_hrs}')    
    
def compute_leave(leave):    
    leave[0] = leave[0] + 8 - df.loc['LA'].sum()
    leave[1] = leave[1] + 4 - df.loc['LS'].sum()
    leave[2] = leave[2] + df.loc['CD'].sum() - df.loc['CN'].sum()
    return leave        

In [167]:
times = np.zeros((8, 14))
df = pd.DataFrame(times, index=labels, columns=dlabels)

### Enter Daily Times Here

In [168]:
# enter leave at start of period

leave = [307.0, 238.0, 0.0]

# update daily times here
# sun0
# mon0
rg(mon0, 8.5)
ln(mon0, 1.0)
# tue0
rg(tue0, 9.0)
# wed0
rg(wed0, 8.5)
ln(wed0, 1.0)
# thu0
rg(thu0, 9.5)
# fri0
rg(fri0, 7.5)
ln(fri0, 1.0)
# sat0
# sun1
# mon1
rg(mon1, 9.0)
# tue1
rg(tue1, 8.0)
ln(tue1, 1.0)
# wed1
# thu1
# fri1
# sat1

In [169]:
# Create Time grid

df_rowsum = pd.concat([df, sum_by_leave_category(df)], axis=1)
df_rowsum.rename(columns={0:'Total'})
df_rowsum2 = df_rowsum.rename(columns={0:'Total'})

df_colsum = df_rowsum2.append(sum_by_days(df),ignore_index=True)
df_colsum.rename(index=row_names, inplace=True)

total_leave = sum_by_leave_category(df).sum()
total_days = sum_by_days(df).sum()
remaining_hrs = 80.0 - total_days

print("\n")
print_by_lv_cat(df)
print(f"total by leave: {total_leave}")
print(f"total by day:   {total_days}")
print(f"remaining to work:   {remaining_hrs}")
print("==================================================================================")
essential = df_colsum.loc[(df_colsum!=0).any(1)]
from IPython.display import display, HTML
display( HTML( essential.style.render() ) )
# essential
print("==================================================================================")
print()
print(f'Leave Balance:           A/L       S/L    Credit')
print("-------------------------------------------------")
print_leave(leave, "Start of Pay Period:")
print_leave_summary(leave)
leave = compute_leave(leave)
print()
print_leave(leave, "End of Pay Perod:")
print_leave_summary(leave)
print("\n")
print_usable_leave(leave)



RG: 60.0
LN: 4.0

total by leave: 64.0
total by day:   64.0
remaining to work:   16.0


,sun0,mon0,tue0,wed0,thu0,fri0,sat0,sun1,mon1,tue1,wed1,thu1,fri1,sat1,Total
RG,0,8.5,9,8.5,9.5,7.5,0,0,9,8,0,0,0,0,60
LN,0,1,0,1,0,1,0,0,0,1,0,0,0,0,4
Total,0,9.5,9,9.5,9.5,8.5,0,0,9,9,0,0,0,0,nan



Leave Balance:           A/L       S/L    Credit
-------------------------------------------------
Start of Pay Period:   307.0     238.0       0.0
                       38.0 d 3.0 hrs     29.0 d 6.0 hrs

End of Pay Perod:      315.0     242.0       0.0
                       39.0 d 3.0 hrs     30.0 d 2.0 hrs


Usable leave: 9.0 d 3.0
